In [1]:
import faiss
import pickle
import numpy as np
import torch

from transformers import CamembertTokenizer, AutoModel
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from typing import List, Tuple

c:\university\project-quant\esg chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1. โหลด FAISS index และ chunk_texts

In [2]:
faiss_index = faiss.read_index("chunk_faiss.index")
with open("chunk_texts.pkl", "rb") as f:
    chunk_texts: List[str] = pickle.load(f)

2. เตรียม WangchanBERTa และฟังก์ชัน encode_sentences_wangchanberta

In [3]:
MODEL_NAME = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = CamembertTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
model = AutoModel.from_pretrained(MODEL_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

CamembertModel(
  (embeddings): CamembertEmbeddings(
    (word_embeddings): Embedding(25005, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): CamembertEncoder(
    (layer): ModuleList(
      (0-11): 12 x CamembertLayer(
        (attention): CamembertAttention(
          (self): CamembertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): CamembertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 

In [4]:
def encode_query(
    query: str,
    max_length: int = 128
) -> np.ndarray:
    """
    แปลงคำถาม (หรือข่าว) เป็น embedding shape = (1, 768)
    """
    enc = tokenizer(
        [query],
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    last_hidden = outputs.last_hidden_state  # (1, L, H)

    mask = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()  # (1, L, H)
    masked_hidden = last_hidden * mask                                         # (1, L, H)

    summed = torch.sum(masked_hidden, dim=1)             # (1, H)
    counts = torch.clamp(mask.sum(dim=1), min=1e-9)       # (1,)
    mean_pooled = summed / counts.unsqueeze(-1)           # (1, H)

    return mean_pooled.cpu().numpy().astype("float32")    # shape = (1, 768)

In [5]:
def encode_sentences_wangchanberta(
    sentences: List[str],
    max_length: int = 128,
    batch_size: int = 16
) -> np.ndarray:
    """
    แปลงลิสต์ประโยค (List[str]) เป็น embedding (mean-pooling) ด้วย WangchanBERTa
    คืน numpy array shape = (N, 768)
    """
    all_embeddings = []
    with torch.no_grad():
        for i in range(0, len(sentences), batch_size):
            batch_sents = sentences[i : i + batch_size]
            enc = tokenizer(
                batch_sents,
                padding=True,
                truncation=True,
                max_length=max_length,
                return_tensors="pt"
            )
            input_ids = enc["input_ids"].to(device)
            attention_mask = enc["attention_mask"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            last_hidden = outputs.last_hidden_state  # (B, L, 768)

            mask = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()  # (B, L, 768)
            masked_hidden = last_hidden * mask                                          # (B, L, 768)

            summed = torch.sum(masked_hidden, dim=1)             # (B, 768)
            counts = torch.clamp(mask.sum(dim=1), min=1e-9)       # (B, 768) เพราะ mask ถูกขยายมิติมา
            # แต่ counts ในที่นี้แต่ละตำแหน่งจะเท่ากับจำนวน token จริง → mean pooling
            mean_pooled = summed / counts                         # (B, 768)

            all_embeddings.append(mean_pooled.cpu().numpy())

    all_embeddings = np.vstack(all_embeddings)  # (N, 768)
    return all_embeddings.astype("float32")


3. encode_query: ใช้ encode_sentences_wangchanberta กับ list ที่มีข้อความเดียว

In [6]:
def encode_query(
    text: str,
    max_length: int = 128
) -> np.ndarray:
    """
    แปลงข้อความ (เช่น ข่าว) เป็น embedding shape = (1, 768)
    โดยเรียกผ่าน encode_sentences_wangchanberta([text])
    """
    embeddings = encode_sentences_wangchanberta(
        sentences=[text],
        max_length=max_length,
        batch_size=1
    )
    # embeddings จะเป็น numpy array shape (1, 768)
    return embeddings


In [7]:
def encode_sentences_wangchanberta(
    sentences: List[str],
    max_length: int = 128,
    batch_size: int = 16
) -> np.ndarray:
    """
    แปลงลิสต์ประโยค (List[str]) เป็น embedding (mean-pooling) ด้วย WangchanBERTa
    คืน numpy array shape = (N, 768)
    """
    all_embeddings = []
    with torch.no_grad():
        for i in range(0, len(sentences), batch_size):
            batch_sents = sentences[i : i + batch_size]
            enc = tokenizer(
                batch_sents,
                padding=True,
                truncation=True,
                max_length=max_length,
                return_tensors="pt"
            )
            input_ids = enc["input_ids"].to(device)
            attention_mask = enc["attention_mask"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            last_hidden = outputs.last_hidden_state  # (B, L, 768)

            mask = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()  # (B, L, 768)
            masked_hidden = last_hidden * mask                                          # (B, L, 768)

            summed = torch.sum(masked_hidden, dim=1)             # (B, 768)
            counts = torch.clamp(mask.sum(dim=1), min=1e-9)       # (B, 768) เพราะ mask ถูกขยายมิติมา
            # แต่ counts ในที่นี้แต่ละตำแหน่งจะเท่ากับจำนวน token จริง → mean pooling
            mean_pooled = summed / counts                         # (B, 768)

            all_embeddings.append(mean_pooled.cpu().numpy())

    all_embeddings = np.vstack(all_embeddings)  # (N, 768)
    return all_embeddings.astype("float32")


4. ฟังก์ชันดึง top-k chunks จาก FAISS (ตรวจสอบมิติให้เป็น (1,768))

In [8]:
def retrieve_top_k_chunks(
    query_embedding: np.ndarray,
    top_k: int = 3
) -> List[Tuple[str, float]]:
    """
    รับ query_embedding (1,768) → คืน list [(chunk_text, distance), ...] top_k
    ตรวจสอบมิติและ dtype ให้ถูกต้องก่อนเรียก faiss.search
    """
    # เปลี่ยนให้เป็น 2 มิติ (n_queries × dim) ถ้า shape ผิด
    # แต่ encode_query จะคืน (1, 768) จึงไม่มีปัญหา
    if query_embedding.ndim == 1:
        query_embedding = query_embedding.reshape(1, -1)
    elif query_embedding.ndim > 2:
        query_embedding = query_embedding.reshape(query_embedding.shape[0], -1)

    # แปลง dtype ให้เป็น float32 (FAISS ต้องการ)
    if query_embedding.dtype != np.float32:
        query_embedding = query_embedding.astype("float32")

    # ตรวจสอบดีบัก 
    #print(">>> Debug: FAISS index dim =", faiss_index.d)               # ควรเป็น 768
    #print(">>> Debug: query_embedding.shape =", query_embedding.shape)  # ควรเป็น (1, 768)

    distances, indices = faiss_index.search(query_embedding, top_k)
    results: List[Tuple[str, float]] = []
    for idx, dist in zip(indices[0], distances[0]):
        results.append((chunk_texts[idx], float(dist)))
    return results

5. ฟังก์ชัน retrieve_context: รวบ context จาก top-k chunks

In [9]:
def retrieve_context(
    news: str,
    top_k: int = 3,
    max_chars: int = 1000
) -> str:
    """
    1. encode ข่าว → embedding shape (1, 768)
    2. retrieve top_k chunks → คืน list of (chunk_text, dist)
    3. รวมแต่ละ chunk (คั่นด้วย '---') จนความยาวรวมไม่เกิน max_chars
    """
    q_emb = encode_query(news, max_length=128)           # (1, 768)
    retrieved = retrieve_top_k_chunks(q_emb, top_k=top_k)

    contexts: List[str] = []
    total_len = 0
    for chunk_text, dist in retrieved:
        if total_len + len(chunk_text) > max_chars:
            break
        contexts.append(chunk_text)
        total_len += len(chunk_text)

    context_str = "\n---\n".join(contexts)
    return context_str

6. เตรียม Ollama (esg-analyzer) และ PromptTemplate ใหม่

In [10]:
llm = Ollama(model="esg-analyzer", temperature=0.0)

C:\Users\User\AppData\Local\Temp\ipykernel_20488\1765551914.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="esg-analyzer", temperature=0.0)


In [11]:
template = """
คุณเป็นผู้ช่วยวิเคราะห์ข่าวหุ้นเชิง ESG ระดับผู้เชี่ยวชาญ

Context from documents:
{context}

News:
{news}

งานของคุณคือ
1. สรุป sentiment ของข่าว (เลือกหนึ่งใน: บวก / กลาง / ลบ) พร้อมเหตุผลอย่างน้อย 2–3 ประโยค
2. วิเคราะห์ว่า “ข่าวนี้จะส่งผลอย่างไรต่อ ESG Rating” ของบริษัทในข่าว 
   ให้ระบุว่า “ESG Rating จะ <สูงขึ้น/ลดลง/ไม่เปลี่ยน>” พร้อมอธิบายเหตุผลอย่างน้อย 2–3 ประโยค
   โดยพิจารณาจากผลกระทบต่อสิ่งแวดล้อม สังคม และธรรมาภิบาล
   หากข่าวไม่เกี่ยวข้องกับ ESG ให้ตอบว่า “ไม่เปลี่ยน” พร้อมเหตุผล
   ลองดูว่า {context} สามารถนำมาช่วยวิเคราะห์ได้หรือไม่ หากได้ให้ใช้ข้อมูลนั้นในการวิเคราะห์ด้วย

**โครงสร้างการตอบ (ตอบครบ 2 หัวข้อ):**

Sentiment: <บวก/กลาง/ลบ>  
เหตุผลสรุป (2–3 ประโยค):  
- …  
- …  

Impact on ESG Rating: <สูงขึ้น/ลดลง/ไม่เปลี่ยน>  
เหตุผลประกอบ (2–3 ประโยค):  
- …  
- …

**ตัวอย่าง (อย่า copy ตรงนี้ ให้โมเดลตอบเฉพาะส่วนโครงสร้างด้านบน):**  
News: ปตท. เตรียมขยายลงทุนสีเขียว เพิ่มสัดส่วนพลังงานหมุนเวียน  
- Sentiment: บวก  
  เหตุผลสรุป (2–3 ประโยค):  
    - ข่าวนี้ชี้ให้เห็นว่าปตท. ให้ความสำคัญกับธุรกิจสีเขียวอย่างจริงจัง  
    - การเพิ่มสัดส่วนพลังงานสะอาดช่วยลดการปล่อยก๊าซเรือนกระจก  
    - ทั้งยังเป็นสัญญาณเชิงบวกต่อผู้ลงทุนที่คำนึง ESG  
- Impact on ESG Rating: สูงขึ้น  
  เหตุผลประกอบ (2–3 ประโยค):  
    - การลงทุนในพลังงานหมุนเวียนจะช่วยยกระดับคะแนนดัชนี ESG ด้านสิ่งแวดล้อมอย่างชัดเจน  
    - โครงการใหม่จะช่วยลดความเสี่ยงเรื่องกฎระเบียบสิ่งแวดล้อม  
    - ทำให้ผู้ถือหุ้นและนักวิเคราะห์มองว่า ESG Profile ของปตท. ดีขึ้น  

**ให้ทำเช่นนี้สำหรับข่าวต่อไปนี้ (ตอบครบตามโครงสร้างด้านบน):**  
{news}
"""

In [12]:
prompt = PromptTemplate(input_variables=["context", "news"], template=template)
chain = LLMChain(llm=llm, prompt=prompt)


C:\Users\User\AppData\Local\Temp\ipykernel_20488\1641642314.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


7. ทดลองใช้งาน RAG + Template

In [13]:
if __name__ == "__main__":
    print("=== RAG + Ollama (esg-analyzer) พร้อมใช้งาน ===")
    news_text = "ปตท.สผ. กระทรวงพลังงานและแร่ธาตุโอมาน และพันธมิตร ลงนามขยายอายุสัญญาโครงการโอมาน แปลง 53"

    # ดึง context จาก PDF (RAG)
    context_str = retrieve_context(news_text, top_k=3, max_chars=1000)
    # print("\n=== Context from PDF ===\n", context_str)

    # ส่ง context + news เข้า LLMChain แล้วแสดงผล
    result = chain.run(context=context_str, news=news_text)
    print("\n=== ผลลัพธ์ที่ได้ ===\n")
    print(result)


=== RAG + Ollama (esg-analyzer) พร้อมใช้งาน ===


C:\Users\User\AppData\Local\Temp\ipykernel_20488\2621994233.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run(context=context_str, news=news_text)



=== ผลลัพธ์ที่ได้ ===

**Sentiment:** กลาง

เหตุผลสรุป (2–3 ประโยค):  
- ข่าวนี้เป็นข้อมูลเชิงบวกต่อธุรกิจพลังงานของปตท. แต่ไม่ได้กล่าวถึงผลกระทบด้านสิ่งแวดล้อมโดยตรง  
- การลงทุนในโครงการโอมานอาจช่วยลดความเสี่ยงเรื่องกฎระเบียบสิ่งแวดล้อม แต่ยังไม่มีผลกระทบเชิงบวกต่อ ESG Rating ทันที  
- ผู้ถือหุ้นและนักวิเคราะห์อาจมองว่าข่าวนี้เป็นปัจจัยบวกที่ช่วยเสริมภาพลักษณ์ของบริษัทในด้านธุรกิจพลังงาน

**Impact on ESG Rating:** ไม่เปลี่ยน

เหตุผลประกอบ (2–3 ประโยค):  
- ข่าวไม่ได้กล่าวถึงการปรับเปลี่ยนพฤติกรรมหรือโครงการใหม่ที่มีผลกระทบเชิงบวกต่อสิ่งแวดล้อมอย่างชัดเจน  
- การลงทุนในโครงการโอมานอาจเป็นปัจจัยบวกที่ช่วยเสริมภาพลักษณ์ของบริษัท แต่ยังไม่ได้กล่าวถึงการปรับเปลี่ยนพฤติกรรมหรือโครงการใหม่ที่มีผลกระทบเชิงบวกต่อ ESG Rating  
- จึงไม่มีผลกระทบเชิงบวกหรือเชิงลบที่ชัดเจนต่อ ESG Rating ของปตท. ในระยะสั้น
